<a href="https://colab.research.google.com/github/riccardomarin/SpectralShapeAnalysis/blob/master/Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

In [ ]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import meshplot
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans

In [ ]:
def plot_pair(v1, v2, f1, f2, c1, c2, color_ops = {}):
    # Compute a scale factor
    M1 = igl.massmatrix(v1, f1, igl.MASSMATRIX_TYPE_VORONOI)
    M2 = igl.massmatrix(v2, f2, igl.MASSMATRIX_TYPE_VORONOI)
    scale_factor = np.sqrt(np.sum(M2)/np.sum(M1))

    # Align the shapes
    v2 = v2 - np.mean(v2,axis=0)
    v1_align = v1 * scale_factor + np.mean(v1,axis=0) + [0.7,-0.7,0.0]

    # Merge the models
    v_all = np.vstack((v1_align, v2))
    f_all = np.vstack((f1, f2 + np.max(f1)+1))
    c_all = np.vstack((c1, c2))
    
    plot(v_all, f_all, c_all, shading = color_ops)

# WKS

We will use the Wave Kernel Signature (WKS) descriptor to do the matching. Recall the formula:

$K_E(x,x) = \sum\limits_{l=1}^{\infty}e^{- \frac{(log(E) - log(\lambda_l))^2}{2\sigma^2}} \phi_l(x)^2 $

Where:
- $sigma = 7 \delta$
- $delta =  (e_{max} - e{min})/ M$
- $e_{max} = log(E_N) - 2\sigma$
- $e_{min} = log(E_1) + 2\sigma$
- $E_N$ is the max eigenvalue in absolute value
- $E_1$ is the min non-zero eigenvalue in absolute value
- $M$ is the number of WKS scales

The tasks are:
- Read the meshes, compute the LBO eigenvectors
- Define the WKS computation
- Visualize the WKS scales on meshes
- Perform the matching using WKS (Nearest-Neighbor in the descriptor space)
- Visualize the matching (and compute the error)

Are the descriptors coherent among the shapes, for different descriptor scales? Is the matching good? We can change the number of descriptors: does it impact the matching?

In [ ]:
v_src, f_src = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_089.off"))

L_src = igl.cotmatrix(v_src, f_src)
M_src = igl.massmatrix(v_src, f_src, igl.MASSMATRIX_TYPE_VORONOI)

evals_src, evecs_src = eigs(L_src,k=100,M=M_src,sigma=0)
evecs_src = evecs_src.astype(np.float32)

v_tar, f_tar = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_043.off"))

L_tar = igl.cotmatrix(v_tar, f_tar)
M_tar = igl.massmatrix(v_tar, f_tar, igl.MASSMATRIX_TYPE_VORONOI)

evals_tar, evecs_tar = eigs(L_tar,k=100,M=M_tar,sigma=0)
evecs_tar = evecs_tar.astype(np.float32)

In [ ]:
def WKS(vertices, Lambda, Phi, Area, wks_size = 100):
    import numpy as np

    n_evecs = Phi.shape[1]
    absoluteEigenvalues = np.abs(Lambda)
    emin = np.log(absoluteEigenvalues[1])
    emax = np.log(absoluteEigenvalues[len(absoluteEigenvalues) - 1])
    
    D = np.matmul(Phi.T, Area.__matmul__(Phi**2))

    s = 7 * (emax - emin) / wks_size
    
    emin = emin + 2 * s
    emax = emax - 2 * s
    es = np.linspace(emin, emax, wks_size)

    T = np.exp(-np.square(
        np.tile(np.reshape(np.log(absoluteEigenvalues), (n_evecs, 1)), (1, wks_size)) - np.tile(es.T, (n_evecs, 1))) / (
                       2 * np.square(s)))
    WKS = np.matmul(D,T)
    WKS = np.dot(Phi, WKS)

    return np.asarray(WKS)

In [ ]:
# Computing the descriptors for the two shapes
D_src = WKS(v_src, evals_src, evecs_src, M_src, wks_size = 200)
D_tar = WKS(v_tar, evals_tar, evecs_tar, M_tar, wks_size = 200)

In [ ]:
# Visualizing descriptors
i = 199
plot_pair(v_src, v_tar, f_src, f_tar, D_src[:,i:i+1], D_tar[:,i:i+1])

In [ ]:
# Nearest Neighbor
treesearch = sp.spatial.cKDTree(D_tar)
p2p = treesearch.query(D_src, k=1)[1]

In [ ]:
# To see the quality of the matching we plot a function on one shape and we transfer it to the other

funz_ = (v_tar - np.min(v_tar,0))/np.tile((np.max(v_tar,0)-np.min(v_tar,0)),(np.size(v_tar,0),1));
colors = np.cos(funz_);
funz_tar = (colors-np.min(colors))/(np.max(colors) - np.min(colors));
funz_src = funz_tar[p2p]


In [ ]:
# subplot(v_src, f_src, funz,s=[1,2,0])
# subplot(v_tar, f_tar, funz2,s=[1,2,1])

plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)

# We can also measure the error
d = igl.exact_geodesic(v_tar, f_tar, 0,p2p)


In [ ]:
d = 0
for i in np.arange(0,6890):
  d = d + igl.exact_geodesic(v_tar, f_tar, np.asarray([i]),np.asarray([p2p[i]]))

d

In [ ]:
def my_WKS(evals, evecs, wks_size = 100):
    abs_evals = np.abs(evals)
    n_evecs = evecs.shape[1]
    
    emin = np.log(abs_evals[1])
    emax = np.log(np.max(abs_evals))
    sigma = 7 * (emax - emin) / wks_size
    
    emin = emin + 2 * sigma
    emax = emax - 2 * sigma
    
    es = np.linspace(emin, emax, wks_size)
    
    for i in np.arange(1, n_evecs):
        squared_evecs = evecs**2
        exponential = np.square(np.exp(es[i]  - np.log(es[i])))/(2 * sigma)
        
    return exponential
    
e = my_WKS(evals_src, evecs_src, 100)
e

# Functional Maps

Here we will use the Functional Maps framework. Given some descriptor $D$ on the first shape and $F$ on the second, we will compute our map $C$ such that: \
$ d = (\phi_{src}^{T} A_{src} D)^T $ \
$ f = (\phi_{tar}^{T} A_{tar} F)^T $ \
$ C d = f $ \
$ C =  f d^{-1} $

In [ ]:
d = (np.matmul(evecs_src.T, np.matmul( M_src.todense() , D_src))).T
f = (np.matmul(evecs_tar.T, np.matmul( M_tar.todense() , D_tar))).T
C = np.linalg.pinv(d) * f

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(C)
plt.colorbar()
plt.show()

In [ ]:
treesearch = scipy.spatial.cKDTree(evecs_src)
p2p = treesearch.query(evecs_tar * C, k=1)[1]

In [ ]:
p2p

In [ ]:
import tensorflow as tf

Ctf = tf.Variable(np.eye(100))
adam = tf.optimizers.Adam(1e-1)

for i in np.arange(0,10000):
  with tf.GradientTape() as grad:
    loss = tf.reduce_sum(tf.square(tf.transpose(tf.matmul(Ctf,d.T)) - f))

dCtf = grad.gradient(loss,Ctf)
adam.apply_gradients(zip([dCtf], [Ctf]))

In [ ]:
plt.imshow(Ctf.numpy())
plt.colorbar()
plt.show()
Ctf